In [ ]:
import numpy as np
A = np.array([[6, 6, 0],
               [3, 0, 1]])
D = np.array([[4,-2,0], [-2,4,0],[0,0,1]])
b = np.array([[3,1]])
c = np.array([-1, 0, 0])
Jbz = [2, 3]
Jb = [2,3]
x = np.array([0, 0.5, 1])
print(solve(D,c,a,Jb,x,Jbz))

[-1.  0.  0.]
Jbz [2, 3]
[0. 0. 0.]
Завершаем работу
[0.2 0.7 0.8]


In [ ]:
import numpy as np
import math

A = np.matrix([[6, 6, 0],
               [3, 0, 1]])
D = np.matrix([[4,-2,0], [-2,4,0],[0,0,1]])
b = np.matrix([[3,1]])
c = np.matrix([[-1], [0], [0]])
J_on = [2, 3]
J_ = [2,3]
x = np.matrix([[0], [0.5], [1]])



def end_method_for_quadratic_task(A, D, b, c, optimal_indexes, right_optimal_indexes, optimal_plan):
    iteration = 0
    while True:
        print('\n------- Итерация № {} -------'.format(iteration))

        a_b_matrix = A[:, optimal_indexes]
        inverse_a_b_matrix = np.linalg.inv(a_b_matrix)

        print('\nБазисная матрица А:\n {}\n'.format(a_b_matrix))
        print('Обратная бащисная матрица А:\n {}\n'.format(inverse_a_b_matrix))

        vector_c_x = c + D * optimal_plan
        print("Вектор с: \n{}".format(vector_c_x))

        c_b = np.matrix([[vector_c_x.item(i)] for i in optimal_indexes])
        print("\nБазисный вектор с: \n{}".format(c_b))

        u = -c_b.transpose() * inverse_a_b_matrix
        print("\nВектор потенциалов u: {}".format(u))

        delta = u * A + vector_c_x.transpose()
        print('\nВектор оценок: {}'.format(delta))

        negative_elements_indexes = get_negative_elements_in_list_indexes(delta.tolist()[0], optimal_indexes)
        print("\nИндексы негативных элементов: {}".format(negative_elements_indexes))

        if len(negative_elements_indexes) == 0:
            break

        j0 = negative_elements_indexes[0]
        print('j0: {}\n'.format(j0))

        delta_j0 = delta.item(j0)
        l = calculate_l(A, right_optimal_indexes, j0, delta_j0, D)
        print("Вектор l: {}".format(l))

        min_theta_index, min_theta = calculate_thetas(l, D, optimal_plan, j0, delta_j0, right_optimal_indexes)
        print("\nj*: {}\n".format(min_theta_index))

        optimal_plan = optimal_plan + min_theta * l.transpose()
        print("\nТекущий оптимальный план:\n {}".format(optimal_plan))

        update_indexes(optimal_indexes, right_optimal_indexes, j0, min_theta_index, inverse_a_b_matrix, A)

        print("\nОпора ограничений: {}".format(optimal_indexes))
        print("\nПравильные оптимальные индексы: {}".format(right_optimal_indexes))

        iteration += 1
    return optimal_plan, optimal_indexes, right_optimal_indexes


def get_negative_elements_in_list_indexes(elements_list, basis_indexes):
    not_basis_indexes = [index for index in range(len(elements_list)) if index not in basis_indexes]
    indexes = [not_basis_indexes[index] for index in range(len(not_basis_indexes))
               if elements_list[not_basis_indexes[index]] < -0.33]
    return indexes


def calculate_l(a_matrix, right_optimal_indexes, j0, delta_j0, d_matrix):
    sign = lambda x: (1, -1)[x < 0]

    elements_count = len(a_matrix.tolist()[0])
    l = [0] * elements_count
    l[j0] = -sign(delta_j0)

    a_b = a_matrix[:, right_optimal_indexes].tolist()

    indexes_count = len(right_optimal_indexes)

    d_b = [[0] * indexes_count for _ in range(indexes_count)]
    d_matrix = d_matrix.tolist()
    i = 0
    j = 0
    for first_index in right_optimal_indexes:
        for second_index in right_optimal_indexes:
            d_b[i][j] = d_matrix[first_index][second_index]
            j += 1

        i += 1
        j = 0

    print("d*: {}".format(d_b))

    h = [[0] * (len(a_b) + indexes_count) for _ in range(len(a_b) + indexes_count)]

    for i in range(len(h)):
        for j in range(len(h[i])):
            if i < indexes_count and j < indexes_count:
                h[i][j] = d_b[i][j]

            if i >= indexes_count > j:
                h[i][j] = a_b[i - indexes_count][j]

            if i < indexes_count <= j:
                h[i][j] = a_b[j - indexes_count][i]

    h = np.matrix(h)
    print("Матрица h:\n {}".format(h))

    b = [0] * indexes_count
    for index in range(indexes_count):
        b[index] = d_matrix[right_optimal_indexes[index]][j0]

    for element in a_matrix[:, j0].tolist():
        b.append(element[0])

    b = np.matrix(b)

    print("\nВектор b: {}".format(b))

    x = -np.linalg.inv(h) * b.transpose()
    print("Вектор x:\n {}".format(x))

    i = 0
    for optimal_index in right_optimal_indexes:
        l[optimal_index] = x.item(i)
        i += 1

    return np.matrix(l)


def calculate_thetas(l, d_matrix, optimal_plan, j0, delta_j0, right_optimal_indexes):
    delta = (l * d_matrix * l.transpose()).item(0)
    print("\ndelta: {}".format(delta))

    thetas = {j0: math.inf if delta == 0 else math.fabs(delta_j0) / delta}

    for optimal_index in right_optimal_indexes:
        thetas[optimal_index] = (-optimal_plan.item(optimal_index) / l.item(optimal_index)
                                 if l.item(optimal_index) < 0 else math.inf)

    min_theta_index = min(thetas, key=thetas.get)
    min_theta = thetas[min_theta_index]

    if min_theta == math.inf:
        raise Exception

    return min_theta_index, min_theta


def update_indexes(optimal_indexes, right_optimal_indexes, j0, min_theta_index, inverse_a_b, a_matrix):
    if j0 == min_theta_index:
        right_optimal_indexes.append(min_theta_index)
        return

    right_optimal_indexes_not_in_optimal_indexes = [index for index in right_optimal_indexes
                                                    if index not in optimal_indexes]

    if min_theta_index in right_optimal_indexes_not_in_optimal_indexes:
        right_optimal_indexes.remove(min_theta_index)

    if min_theta_index in optimal_indexes:
        s = optimal_indexes.index(min_theta_index)

        exists = False
        j_plus = -1
        for index_not_in_optimal_indexes in right_optimal_indexes_not_in_optimal_indexes:
            value = inverse_a_b * a_matrix[:, index_not_in_optimal_indexes]

            if value.item(s) != 0:
                exists = True
                j_plus = index_not_in_optimal_indexes
                break

        if exists:
            right_optimal_indexes.remove(min_theta_index)
            optimal_indexes[s] = j_plus

            return

        optimal_indexes[s] = j0
        min_theta_index_in_right_optimal_indexes = right_optimal_indexes.index(min_theta_index)
        right_optimal_indexes[min_theta_index_in_right_optimal_indexes] = j0


x, J_on, J_ = end_method_for_quadratic_task(A, D,
                                            b, c, J_on,
                                            J_,
                                            x)

print("\nКонечный метод:")
print("Оптимальный план:\n {}\nОпорные индексы {}\nПравильные оптимальные индексы: {}".format(x.transpose(),
                                                                                  J_on,
                                                                                  J_))


------- Итерация № 0 -------


IndexError: ignored

In [ ]:
print(np.linalg.matrix_rank(a))

2


In [ ]:
def solve(D,c,a,Jb,x,Jbz):
  while True:
    cx = c + np.dot(D, x)
    for i in range(len(cx)):
      if abs(cx[i]) < 10 ** -6:
        cx[i] = 0
    cbx = [cx[i - 1] for i in Jb]
    Ab = np.zeros((len(a), len(Jb)))
    for i in range(len(Jb)):
      Ab[:, i] = a[:, Jbz[i] - 1]
    Ab_inv = np.linalg.inv(Ab)
    ux = np.dot(np.negative(cbx), Ab_inv)
    deltax = np.dot(ux, a) + cx
    for i in range(len(deltax)):
      if abs(deltax[i]) < 10**-6:
        deltax[i] = 0
    print(deltax)
    if np.min(deltax) >= 0:
      print('Завершаем работу')
      return x
    for i in range(len(deltax)):
      if deltax[i] < 0:
        j0 = i
        break
    Jnb = []
    for i in range(1, len(a[0]) + 1):
      if i not in Jbz:
        Jnb.append(i)
    Jnb = np.array(Jnb)
    l = [0 for _ in range(len(a[0]))]
    for i in range(len(Jnb)):
      if Jnb[i] - 1 == j0:
        l[Jnb[i] - 1] = 1
    d = []
    print('Jbz', Jbz)
    for i in range(len(D)):
      d.append([])
      for j in range(len(D)):
        if (i + 1) in Jbz and (j + 1) in Jbz:
          d[-1].append(D[i][j])
    i = 0
    while i < len(d):
      if len(d[i]) == 0:
        d.pop(i)
      else:
        i += 1
    d = np.array(d)
    Abz = np.zeros((len(a), len(Jbz)))
    for i in range(len(Jbz)):
      Abz[:, i] = a[:, Jbz[i] - 1]
    H = np.block([[d, np.transpose(Abz)], [Abz, np.zeros((2, 2))]])
    H_inv = np.linalg.inv(H)
    bz = []
    for i in Jbz:
      bz.append(D[j0][i - 1])
    for i in range(len(a)):
      bz.append(a[i][j0])
    xx = np.dot(np.negative(H_inv), bz)
    for i in range(len(Jbz)):
      l[Jbz[i] - 1] = xx[i]
    sigma = np.dot(np.dot(l, D), l)
    tetta = [99999999 for _ in range(len(a[0]))]
    tetta[j0] = 99999999 if sigma == 0 else abs(deltax[j0]) / sigma
    for i in range(len(a[0])):
      if i + 1 in Jbz:
        if l[i] < 0:
          tetta[i] = -(x[i] / l[i])
    min, pi = 99999999, 99
    for i in range(len(tetta)):
      if tetta[i] < min:
        min, pi = tetta[i], i
    if min == 99999999:
      print('Целевая функция не ограничена снизу на множестве допустимых планов')
      return
    x = x + np.dot(min, l)
    f1 = np.setdiff1d(Jbz, Jb)
    fl = 0
    if pi == j0:
      Jbz = np.append(Jbz, j0 + 1)
    elif pi + 1 in f1:
      Jbz = Jbz[Jbz != pi + 1]
    else:
      if pi + 1 in Jb:
        for i in range(len(Jb)):
          if Jb[i] == pi + 1:
            s = i
            break
        for i in f1:
          q = np.dot(Ab_inv, a[:, i-1])
          if q[s] != 0:
            for ii in range(len(Jb)):
              if Jb[ii] == pi + 1:
                Jb[ii] = i
            Jbz = Jbz[Jbz != pi + 1]
            fl = 1
            break
      if fl == 0:
        if pi + 1 in Jb:
          for i in range(len(Jb)):
            if Jb[i] == pi + 1:
              s = i
              break
          for i in f1:
            q = np.dot(Ab_inv, a[:, i - 1])
          if q[s] == 0:
            for ii in range(len(Jb)):
              if Jb[ii] == pi + 1:
                Jb[ii] = j0 + 1
            for ii in range(len(Jbz)):
              if Jbz[ii] == pi + 1:
                Jbz[ii] = j0 + 1
          if np.array_equal(Jb, Jbz):
            for ii in range(len(Jb)):
              if Jb[ii] == pi + 1:
                Jb[ii] = j0 + 1
            for ii in range(len(Jbz)):
              if Jbz[ii] == pi + 1:
                Jbz[ii] = j0 + 1




In [ ]:
def solve(D,c,a,Jb,x,Jbz):
  while True:
    cx = c + np.dot(D, x) # одиночный вектор даблов
    for i in range(len(cx)): # инт
      if abs(cx[i]) < 10 ** -6: # присвоение даблу с обоссаной точностью нулю
        cx[i] = 0
    cbx = [cx[i - 1] for i in Jb] # вектор даблов  базисный cx
    Ab = np.zeros((len(a), len(Jb))) # инициализация базисной матрицы
    for i in range(len(Jb)):
      Ab[:, i] = a[:, Jbz[i] - 1] # присвоение столбцу базисного индекса
    Ab_inv = np.linalg.inv(Ab) # обратная матрица от базисной
    ux = np.dot(np.negative(cbx), Ab_inv) # вектор даблов ux
    deltax = np.dot(ux, a) + cx # вектор даблов delta x
    for i in range(len(deltax)):
      if abs(deltax[i]) < 10**-6:
        deltax[i] = 0 # если кастрированная точность то ставишь 0 10 ^ -16 например
    print(deltax)
    if np.min(deltax) >= 0: # если все положительные в делта икс то завершаешь работу возвращаешь x
      print('Завершаем работу')
      return x
    for i in range(len(deltax)):
      if deltax[i] < 0:
        j0 = i # запоминмаешь индекс отрицательной компоненты deltax
        break
    print('asdasd')
    Jnb = [] # вектор интов небазисные индексы
    for i in range(1, len(a[0]) + 1):
      if i not in Jbz:
        Jnb.append(i)
    Jnb = np.array(Jnb) # ну небазисные индексы
    print('Jnb', Jnb)
    l = [0 for _ in range(len(a[0]))] # вектор даблов состоит из нулей стольки сколько столбцов в матрице а 
    for i in range(len(Jnb)):
      if Jnb[i] - 1 == j0: # в ту позицию где стоит j0 ставишь 1
        l[Jnb[i] - 1] = 1
    d = [] # матрица даблов d*
    for i in range(len(D)):
      d.append([])
      for j in range(len(D)):
        if (i + 1) in Jbz and (j + 1) in Jbz:
          d[-1].append(D[i][j]) # добавляешь все элементы у которых i и j принадлежит Jb*
    print('d', d)
    i = 0
    while i < len(d):
      if len(d[i]) == 0:
        d.pop(i)
      else:
        i += 1
    # сносишь пустые кастраты в виде пустых массивов
    d = np.array(d)
    print('jbz', Jbz)
    Abz = np.zeros((len(a), len(Jbz)))
    for i in range(len(Jbz)):
      Abz[:, i] = a[:, Jbz[i] - 1]
    print('abz', Abz)
    # базисная матрица из столбцов Jb*
    print(d, Abz)
    H = np.block([[d, np.transpose(Abz)], [Abz, np.zeros((2, 2))]])
    print('H', H)
    # делаешь матрицу H
    H_inv = np.linalg.inv(H)
    print('H_inv', H_inv)
    # обратная матрица H
    bz = []
    for i in Jbz:
      bz.append(D[j0][i - 1])
    for i in range(len(a)):
      bz.append(a[i][j0])
    # вектор b со звездой
    xx = np.dot(np.negative(H_inv), bz) # битый вектор x
    for i in range(len(Jbz)):
      l[i] = xx[i] # вектор l'
    print('l', l)
    sigma = np.dot(np.dot(l, D), l) # ну сигма и сигрма
    tetta = [99999999 for _ in range(len(a[0]))] # инициализируешь бесконечностями вектор tetta
    tetta[j0] = 99999999 if sigma == 0 else abs(deltax[j0]) / sigma # tettaj0 по формуле
    for i in range(len(a[0])):
      if i + 1 in Jbz:
        if l[i] < 0:
          tetta[i] = -(x[i] / l[i])
    # по формулам дальше tetta
    min, pi = 99999999, 99
    for i in range(len(tetta)):
      if tetta[i] < min:
        min, pi = tetta[i], i
    # минималки из tett
    if min == 99999999:
      print('Целевая функция не ограничена снизу на множестве допустимых планов')
      return
    x = x + np.dot(min, l)
    # увеличиваешь x
    #
    fl=0 
    f1 = np.setdiff1d(Jbz, Jb)
#вервый случай ифа
    
    if pi == j0:
      Jbz = np.append(Jbz, j0)
      print("первый случай")
    elif pi + 1 in f1:
      Jbz = Jbz[Jbz != pi + 1] # второй случай
      print("второй случай")
    else: # третий
      if pi + 1 in Jb:
        for i in range(len(Jb)):
          if Jb[i] == pi + 1:
            s = i
            break
        for i in f1:
          q = np.dot(Ab_inv, a[:, i-1])
          if q[s] != 0:
            for ii in range(len(Jb)):
              if Jb[ii] == pi + 1:
                Jb[ii] = i
            Jbz = Jbz[Jbz != pi + 1]
            fl = 1
            break
      if fl == 0: # четвертый
        if pi + 1 in Jb:
          for i in range(len(Jb)):
            if Jb[i] == pi + 1:
              s = i
              break
          for i in f1:
            q = np.dot(Ab_inv, a[:, i - 1])
          if q[s] == 0:
            for ii in range(len(Jb)):
              if Jb[ii] == pi + 1:
                Jb[ii] = j0 + 1
            for ii in range(len(Jbz)):
              if Jbz[ii] == pi + 1:
                Jbz[ii] = j0 + 1
          if np.array_equal(Jb, Jbz):
            for ii in range(len(Jb)):
              if Jb[ii] == pi + 1:
                Jb[ii] = j0 + 1
            for ii in range(len(Jbz)):
              if Jbz[ii] == pi + 1:
                Jbz[ii] = j0 + 1

In [ ]:
cx = c + np.dot(D, x)
for i in range(len(cx)):
  if abs(cx[i]) < 10 ** -6:
    cx[i] = 0
print(cx)

[[ 2]
 [ 4]
 [-6]]


In [ ]:
cbx = [cx[i - 1] for i in Jb]
Ab = np.zeros((len(a), len(Jb)))
for i in range(len(Jb)):
  Ab[:, i] = a[:, Jbz[i] - 1]
print(Ab)
Ab_inv = np.linalg.inv(Ab)
ux = np.dot(np.negative(cbx), Ab_inv)

print(ux)

[[0. 1.]
 [1. 0.]]


ValueError: ignored

In [ ]:
deltax = np.dot(ux, a) + cx
for i in range(len(deltax)):
  if abs(deltax[i]) < 10**-6:
    deltax[i] = 0
print(Jbz, deltax)

[1 2 4] [0.  0.  0.2 0. ]


In [ ]:
if np.min(deltax) >= 0:
  print('Завершаем работу')

Завершаем работу


In [ ]:
for i in range(len(deltax)):
  if deltax[i] < 0:
    j0 = i
    break
print(j0)

3


In [ ]:
Jnb = []
for i in range(1, len(a[0]) + 1):
  if i not in Jbz:
    Jnb.append(i)
Jnb = np.array(Jnb)
l = [0 for _ in range(len(a[0]))]
for i in range(len(Jnb)):
  if Jnb[i] - 1 == j0:
    l[Jnb[i] - 1] = 1
print(l)

[0, 0, 0, 1]


In [ ]:
d = []
print(Jbz)
for i in range(len(D)):
  d.append([])
  for j in range(len(D)):
    if (i + 1) in Jbz and (j + 1) in Jbz:
      print(i, j)
      d[-1].append(D[i][j])

i = 0
while i < len(d):
  if len(d[i]) == 0:
    d.pop(i)
  else:
    i += 1
print(d)
d = np.array(d)
Abz = np.zeros((len(a), len(Jbz)))
print(len(Jbz))
print(Abz)
for i in range(len(Jbz)):
  Abz[:, i] = a[:, Jbz[i] - 1]
print(Abz)
print('asd')
print(d)
H = np.block([[d, np.transpose(Abz)], [Abz, np.zeros((2, 2))]])
print(H)
H_inv = np.linalg.inv(H)
print(H_inv)


[1 2]
0 0
0 1
1 0
1 1
[[2, 1], [1, 1]]
2
[[0. 0.]
 [0. 0.]]
[[1. 0.]
 [0. 1.]]
asd
[[2 1]
 [1 1]]
[[2. 1. 1. 0.]
 [1. 1. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
[[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 1. -0. -2. -1.]
 [ 0.  1. -1. -1.]]


In [ ]:
bz = []
for i in Jbz:
  bz.append(D[j0][i - 1])
print(len(a))
for i in range(len(a)):
  bz.append(a[i][j0])
print(bz)

2
[0, 0, 1, 2]


In [ ]:
xx = np.dot(np.negative(H_inv), bz)
print(xx)

[-1. -2.  4.  3.]


In [ ]:

for i in range(len(Jbz)):
  l[Jbz[i] - 1] = xx[i]

print(l)

[-1.0, -2.0, 0, 1]


In [ ]:
sigma = np.dot(np.dot(l, D), l)
print(sigma)

10.0


In [ ]:
tetta = [99999999 for _ in range(len(a[0]))]
tetta[j0] = 99999999 if sigma == 0 else abs(deltax[j0]) / sigma
for i in range(len(a[0])):
  if i + 1 in Jbz:
    if l[i] < 0:
      tetta[i] = -(x[i] / l[i])

print(tetta)

[1.75, 1.25, 99999999, 0.05]


In [ ]:
min, pi = 99999999, 99
for i in range(len(tetta)):
  if tetta[i] < min:
    min, pi = tetta[i], i
if min == 99999999:
  print('Целевая функция не ограничена снизу на множестве допустимых планов')
print(min, pi)

0.05 3


In [ ]:
x = x + np.dot(min, l)

In [ ]:
print(x)

[1.7 2.4 0.  0.3]


In [ ]:
if pi == j0:
  print('1ый исход')
  Jbz = np.append(Jbz, j0 + 1)

1ый исход


In [ ]:
print(Jbz, Jb)

[1 2 3] [1 2]


In [ ]:
f1 = np.setdiff1d(Jbz, Jb)
print(f1)

[3]


In [ ]:
if pi + 1 in f1:
  print('2-ой исход')
  Jbz = Jbz[Jbz != pi + 1]
  print(Jbz)

2-ой исход
[1 2]


In [ ]:
if pi + 1 in Jb:
  for i in range(len(Jb)):
    if Jb[i] == pi + 1:
      s = i
      break
  for i in f1:
    q = np.dot(Ab_inv, a[:, i - 1])
    if q[s] == 0:
      print('4-ый исход')
      for ii in range(len(Jb)):
        if Jb[ii] == pi + 1:
          Jb[ii] = j0 + 1
      for ii in range(len(Jbz)):
        if Jbz[ii] == pi + 1:
          Jbz[ii] = j0 + 1
  if np.array_equal(Jb, Jbz):
    print('4-ый исход')
    print('sad')
    print(pi)
    for ii in range(len(Jb)):
        if Jb[ii] == pi + 1:
          Jb[ii] = j0 + 1
    for ii in range(len(Jbz)):
      if Jbz[ii] == pi + 1:
        Jbz[ii] = j0 + 1
print(Jb, Jbz)

  
    
  

[1 2] [1 2 3]


In [ ]:
if pi + 1 in Jb:
  for i in range(len(Jb)):
    if Jb[i] == pi + 1:
      s = i
      break
  for i in f1:
    q = np.dot(Ab_inv, a[:, i-1])
    if q[s] != 0:
      for ii in range(len(Jb)):
        if Jb[ii] == pi + 1:
          Jb[ii] = i
      Jbz = Jbz[Jbz != pi + 1]
      break

In [ ]:
if pi + 1 in Jb:
  for i in range(len(Jb)):
    if Jb[i] == pi + 1:
      s = i
      break
  for i in f1:
    q = np.dot(Ab_inv, a[:, i - 1])
    if q[s] == 0:
      print('4-ый исход')
      for ii in range(len(Jb)):
        if Jb[ii] == pi + 1:
          Jb[ii] = j0 + 1
      for ii in range(len(Jbz)):
        if Jbz[ii] == pi + 1:
          Jbz[ii] = j0 + 1
  if np.array_equal(Jb, Jbz):
    print('4-ый исход')
    print('sad')
    print(pi)
    for ii in range(len(Jb)):
        if Jb[ii] == pi + 1:
          Jb[ii] = j0 + 1
    for ii in range(len(Jbz)):
      if Jbz[ii] == pi + 1:
        Jbz[ii] = j0 + 1
print(Jb, Jbz)

  
    
  

[1 2] [1 2 3]
